# Compose materials from structures in CIF files and databases.

Beyond the simplest materials, most crystal structures are too complicated to enter manually. For that purpose, NCrystal v3.6 introduced support for reading crystal structures from other existing sources. This mostly means CIF files (https://www.iucr.org/resources/cif), but also from select online databases, or even a plethora of other input formats (e.g. Quantum Espresso files) through the impressive conversion capabilities of the ASE framework (https://wiki.fysik.dtu.dk/ase/).


## Preamble ##
Fix dependencies and tune jupyter a bit. Feel free to replace as you wish:

In [ ]:
import pathlib
import os
if 'TSL_SCHOOL_DIR' in os.environ:
    if any( (p/".git").is_dir() for p in (pathlib.Path(".").absolute().resolve()/"dummy").parents ):
        raise RuntimeError('Please copy notebook to a work directory')

In [ ]:
#Uncomment to get dependencies via pip: !pip install --quiet ipympl matplotlib spglib gemmi
#Also add "pymatgen' to the list, if you will be using materials from the Materials Project.

In [ ]:
%matplotlib ipympl

In [ ]:
import matplotlib
matplotlib.rcParams.update({"figure.figsize":(6.4*0.5,4.8*0.5),"figure.dpi":150,'font.size':5})

In [ ]:
%%html
<style>div.jupyter-widgets.widget-label {display: none;}</style>

Always import NCrystal of course:

In [ ]:
import NCrystal as NC
assert NC.version_num >=  3005081
NC.test() #< quick unit test that installation works!

The following environment variable is not strictly necessary, but included here to prevent repeated downloads of the same files from online databases:

In [ ]:
import os
os.environ['NCRYSTAL_ONLINEDB_CACHEDIR'] = './ncrystal_onlinedb_filecache'

## Basic CIF data support
Let us start by a simple case, where somehow we have gotten a piece of very simple CIF data already defined directly in our Python code:

In [ ]:
dummy_cif_data="""
data_al
loop_
_space_group_IT_number           225
_symmetry_space_group_name_Hall  '-F 4 2 3'
_symmetry_space_group_name_H-M   'F m -3 m'
_cell_angle_alpha                90
_cell_angle_beta                 90
_cell_angle_gamma                90
_cell_length_a                   4.049
_cell_length_b                   4.049
_cell_length_c                   4.049
_cell_volume                     66.381
loop_
_atom_site_label
_atom_site_fract_x
_atom_site_fract_y
_atom_site_fract_z
Al 0.00000 0.00000 0.00000
"""

The NCMATComposer can load this directly, and we can indeed once recognise see the aluminium structure that we have also used in previous examples:

In [ ]:
c_cif_Al = NC.NCMATComposer(dummy_cif_data)
print(c_cif_Al())

## A real-world example: Bismuth from CIF

Or, here is a more complicated example, where we have found an entry in the online Crystallography Open Database by using the search functionality at https://www.crystallography.net/cod/. Specifically we have searched for Bismuth and found the entry https://www.crystallography.net/cod/7123352.html (the number 7123352 is the ID of the material in the database). Now, we could in principle use the web interface and download the .cif file manually, but NCrystal can do this automatically if you simply specify the entry in a string as `"codid::7123352"` (btw., we also support the online database at https://materialsproject.org/ where entries must be encoded like `"mpid::87"`, but access requires an API key so the examples are not suitable for this Notebook) **COMMENT STILL NEEDED?**:

In [ ]:
c_cif_codBi_attempt1 = NC.NCMATComposer('codid::7123352')
print(c_cif_codBi_attempt1())

As you can tell, this seemed to work, but there was several warnings emitted along the way (be sure to scroll to the bottom of the previous output cell if you missed them). For reference, these warnings are also captured and repeated inside the NCMAT data. We can look at the warnings one at a time, however not exactly in the order they were emitted.

The warning `"Fractional coordinate (0.666667,0.333333,0.56724) interpreted as special position (0.666666666666667,0.333333333333333,0.56724) to avoid numerical precision issues"`: Basically, the precision in the input data was too low, and NCrystal detected this and corrected for it. That looks fine and nothing to worry about.

The warning `"ignoring uiso info present in CIF input for "Bi" since uiso_temperature parameter value is not provided"` is emitted since the CIF data actually DID contain information about atomic displacements, but NCrystal also needs to know the temperature value for which these atomic displacements were measured (or modelled or whatever). To figure that out, we will have to try to inspect the raw CIF data and see if we can find a clue - either directly in the data itself or by pursuing the references given therein. We will get back to that.

The warning `"Anisotropic displacement for Bi ignored (u11=0.01993, u12=0.00996, u13=0, u22=0.01993, u23=0, u33=0.01684)"` means that not only did the CIF file contain atomic displacement info, it even contained *anisotropic* displacement info. Using these is currently not supported by NCrystal, and in practice their effect is likely negligible for most applications. Thus, this warning is just there as a notification, there is nothing to be done about it.

Finally, the warning `"SG-166 available in multiple choices ("R -3 m:H", "R -3 m:R") and which one was not encoded explicitly in the _space_group_name_H-M_alt CIF field. Consider overriding the space group explicitly when loading this file."` is because the space group symmetry could not be determined unambiguously from the file. This happens often, since many of the 230 space groups have more than one "setting, as we can see in the following list:

In [ ]:
import gemmi
for num,xhm in sorted((t.number,t.xhm()) for t in gemmi.spacegroup_table()):
    print(f'SG-{str(num).ljust(6)}: "{xhm}"')


Let us address these issues one by one. First of all, we can inspect the raw CIF data, as well as consult the reference paper at https://dx.doi.org/10.1039/C9CC02820J (paywall) which is mentioned in the raw CIF data, and this mention which was also transferred automatically into the generated NCMAT data above. The paper is behind a paywall, but reading it one discovers that they considered two temperatures, 100K and 298K. Now, let us look at the raw CIF data. This is most easily done by using the `CIFSource` object from the `NCrystal.cifutils` module:

In [ ]:
import NCrystal.cifutils as nccif
cifsrc = nccif.CIFSource('codid::7123352')
print( cifsrc.load_data() )

In case you missed it above, we found a single mention of temperature in the line: 
```
_cod_data_source_block           300K
```
Which is very close to one of the two values (100K and 298K) we found in the paper. So we will for simplicity assume the atomic displacements in the file, are valid at 299K.

Concerning which of the spacegroups to use, `"R -3 m:H"` or `"R -3 m:R"`, we will have to simply see the effect. Let us try to create the materials again. Since we need to pass along a few more parameters, we will use the `NCMATComposer.from_cif(..)` method, rather than the constructor `NCMATComposer(..)`:

In [ ]:
c_choice1 = NC.NCMATComposer.from_cif('codid::7123352', uiso_temperature=299, override_spacegroup="R -3 m:H")

In [ ]:
#This second one is going to fail, so enclosing in try/except to not break the entire notebook:
try:
    c_choice2 = NC.NCMATComposer.from_cif('codid::7123352', uiso_temperature=299, override_spacegroup="R -3 m:R")
except NC.NCBadInput as e:
    print(f'ERROR NCBadInput: {e}')

Well, in this case one of the two possible spacegroup settings turned out to be incompatible with the unit cell parameters defined in the same CIF file, so that means we are left with the first choice, `"R -3 m:H"`. Let us (for convenience) give the new material a virtual name, and inspect the beautiful NCMAT data we have thus created:

In [ ]:
c_choice1.register_as('newBi.ncmat')
print(c_choice1())

We can compare that with the Bismuth already in NCrystal:

In [ ]:
print(NC.createTextData("stdlib::Bi_sg166.ncmat").rawData)

In [ ]:
import NCrystal.plot as ncplot
ncplot.plot_xsects(c_choice1,"stdlib::Bi_sg166.ncmat")

Indeed the results look very similar, except for the fact that the stdlib file contains a full VDOS rather than just a simplified one:

In [ ]:
info_std = NC.createInfo("stdlib::Bi_sg166.ncmat")
di_std = info_std.dyninfos[0]
info_new = c_choice1.load().info
di_new = info_new.dyninfos[0]
ncplot.plot_vdos(di_std,di_new)

For fun, we can compare the atomic displacements predicted by the two VDOS curves:

In [ ]:
print( di_new.correspondingAtomInfo().msd )
print( di_std.correspondingAtomInfo().msd )

Pretty close, all things considered!! So we can be pretty happy with our new NCMAT data for Bismuth. A possible improvement would of course be to add a phonon DOS curve for it, which we will get back to in another notebook.

But one question might bother you: How would I verify that I loaded the right crystal structure if I did not have a "stdlib::Bi_sg166.ncmat" to compare with. Well, first of all you could have looked at the structure in the resulting material and tried to read about the structure in the references (or in online search results), and perhaps you would have been satisfied. Barring an experimental campaign at a powder diffractometer, there is one other approach which fortunately is possible for most materials: Find not just 1, but many, entries for the material in online databases, and see how they compare after loading. Perhaps load each of them like above, and finally use `NCrystal.plot.plot_xsects` to check that the Bragg edges all line up. But there is one other quick-and-dirty way, which is to ask NCrystal to compare the crystal structure in a given NCMAT file, with any online database entries references in the comments of that NCMAT file. For instance, we saw above that `"stdlib::Bi_sg166.ncmat"` contained a total of 5 such mentions of COD entries, and we can easily check them like this:

In [ ]:
import NCrystal.cifutils as nccif
nccif.produce_validation_plot('stdlib::Bi_sg166.ncmat',line_width_scale=0.2,quiet=True)

Looks pretty good! For reference, we can of course also make such a plot without having to edit the comments of a CIF file. For instance after searching COD for relevant Bismuth structures, you can write the following code:

In [ ]:
import NCrystal.plot
bismuth_srcs = ['newBi.ncmat']
for codid in [2310889,5000215,7123351,7123352,9008576]:
    c=NC.NCMATComposer(f'codid::{codid}',plotlabel=f'codid::{codid}',quiet=True)
    c.transfer_dyninfo_objects('stdlib::Bi_sg166.ncmat')
    bismuth_srcs.append( c )
NC.plot.plot_xsects(*bismuth_srcs,extra_cfg='comp=bragg')

## Site occupancy factors
Many real-world crystals have a defect where some sites are not 100% occupied. Accordingly, many CIF files contain so-called "site occupancy factors", with values less than 1.0. The python layer of NCrystal (including the `NCMATComposer` and `CIFLoader`) supports these, but currently (NCrystal v3.6) the support has not made it into the underlying C++ backend yet. So for now, the solution is that the `NCMATComposer` carefully uses the support for overriding atomic data to insert a bit of "dark matter" into the crystal, with the right masses and mixtures of course. Let us see a silly fabricated example:

In [ ]:
cif_data_with_siteoccu = """data_DUMMY
_cell_length_a 4.0
_cell_length_b 4.0
_cell_length_c 4.0
_cell_angle_alpha 90.0
_cell_angle_beta  90.0
_cell_angle_gamma 90.0
_space_group_name_H-M_alt 'P m -3 m'
loop_
_atom_site_label
_atom_site_type_symbol
_atom_site_occupancy
_atom_site_fract_x
_atom_site_fract_y
_atom_site_fract_z
C  C  0.88 0.0 0.0 0.0
Al Al 1.00 0.5 0.5 0.5
"""

If we load it, we get (scroll down and observe the hack in the `@ATOMDB` section):

In [ ]:
c_siteoccu = NC.NCMATComposer.from_cif(cif_data_with_siteoccu)
print( c_siteoccu() )

The material works:

In [ ]:
c_siteoccu.plot_xsect()

The only problem is that asking about composition, density, or cross-section-per-atom will give misleading results. However, the results cancel out, so the cross-section-per-volume is correct!

In [ ]:
mat_siteoccu = c_siteoccu.load()
import pprint
pprint.pprint(mat_siteoccu.info.composition)
mat_siteoccu.plot(xsmode='macroscopic')

The takeaway for using NCMATComposer or CIFLoader for materials with non-unitary site occupancies:
* You can trust most results if you load and use the material in Python, but be aware of densities, composition, and per-atom cross sections.
* You can trust the results if using McStas (incl. McStasScript) because NCrystal provides the whole physics in terms of macroscopic cross sections.
* You can **NOT** use such materials in OpenMC or Geant4: they will likely fail already during initialisation when asked to prepare data with the element "Og" (Z=118).
* In the "near future" we will extend the support, to make sure such materials can be used everywhere with no issues.

## Online databases
In addition to the Crystallography Open Database (COD) at https://www.crystallography.net/cod/, NCrystal currently has built-in support for one other database, namely the Materials Project at https://materialsproject.org/ which you can reference like "mpid::23231" (for the entry  https://www.materialsproject.org/materials/mp-23231). Unlike the very straight-forward usage of COD, there are a few things to be aware off when accessing the Materials Project:
* CIF data is generated on-the-fly on the Materials Project, so it might come in a variety of "settings" (e.g. low-precision vs. high-precision numbers, or primitive versus conventional cell). So to make sure you get a CIF file which is most appropriate for NCrystal, **do not download the CIF data from the website manually**. Instead, use the website to find the material ID (say, `23231`), and the use the string `mpid::23231` in place of a path to the CIF file in your NCrystal code.
* Access requires an API key. It is simple to get, just sign in to the materialsproject.org site (using e.g. your GitHub, google or facebook acount), and access your dashboard by clicking the small logo in the upper right corner. On the dashboard page you can see you API key, which is a (random looking) string of 32 characters. If you put that string into the environment variable named `MATERIALSPROJECT_USER_API_KEY`, then the access should work just as it does for COD. You can do that by putting an `export` statement in your `.bashrc`, or in whatever way is appropriate for your system. If all else fail, you can do it directly in Python:

In [ ]:
import os
os.environ['MATERIALSPROJECT_USER_API_KEY'] = 'your32charkeygoeshere'

Finally, as done at the top of this notebook, note that you can avoid having NCrystal repeatedly download the same CIF files again and again from the online databases, if you set the following environment variable:

In [ ]:
import os
os.environ['NCRYSTAL_ONLINEDB_CACHEDIR'] = './ncrystal_onlinedb_filecache'